### 벅스뮤직 실시간 차트 테이터 가져오기
- 웝 크롤링 기본적인 방법으로 데이터 연결(requests)

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import sys
import pandas as pd

In [53]:
url='https://music.bugs.co.kr/chart/track/realtime/total'
bugs_html=requests.get(url)

if bugs_html.status_code != 200:
    sys.exit('웹 페이지 정보를 가져올수 없습니다.')

bugs_soup=bs(bugs_html.content, 'html.parser')
bugs_soup=bugs_soup.find('tbody')
tr_soup=bugs_soup.find_all('tr')  # 리스트형 구조로 'tr' 태그 전체 찾아오기

bugs_list=[]
# 순위, 곡명, 아티스트, 앨범
for tr in tr_soup:
    rank=tr.find('strong').text # 순위 값 추출
    title=tr.find('p', class_='title').text.replace("\n","") #곡 제목 추출
    artist=tr.find('p', class_='artist').text.replace("\n","") #가수 이름 추출
    album=tr.find('a', class_='album').text.replace("\n","")     #앨범명 추출
    bugs_list.append({"순위":rank, "곡명":title, "가수":artist, '앨범':album})

df1=pd.DataFrame(bugs_list)
df1

,순위,곡명,가수,앨범
0,1,Permission to Dance,방탄소년단,Butter / Permission to Dance
1,2,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집
2,3,Bad Habits,Ed Sheeran(에드 시런),Bad Habits
3,4,Weekend,태연 (TAEYEON),Weekend
4,5,Next Level,aespa,Next Level
...,...,...,...,...
95,96,좋아 좋아,일기예보,좋아좋아
96,97,이렇게 좋아해 본 적이 없어요,CHEEZE(치즈),이렇게 좋아해 본 적이 없어요 (소녀의 세계 X CHEEZE(치즈))
97,98,큰일이다,V.O.S.,Routine Free
98,99,잠이 오질 않네요,장범준,잠이 오질 않네요


### 멜론 차트 정보 가져오기
- selenium 모듈 사용
- selenium 모듈은 chromedriver.exe를 이용해 크롬 부라우저를 직접 컨트롤
- 크롬 드라이버 설치:https://chromedriver.chromium.org/downloads

In [32]:
#!pip install selenium

In [36]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import sys
import pandas as pd
import time

In [86]:
driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
time.sleep(2)  # 대기시간(초)
url='https://www.melon.com/chart/index.htm' # chart,/ dat,/ week,/ month
driver.get(url)       # 실행된 chromedriver 주소창에 url 입력 후 이동
driver.implicitly_wait(10) #사이트 화면이 나타날떄까지 기다림(최대10초)

melon_html=driver.page_source #chromedriver 화면 전체 소스코드 가져오기





melon_soup=bs(melon_html, 'html.parser')
melon_soup=melon_soup.find('tbody')
tr_soup=melon_soup.find_all('tr')

melon_list=[]
cnt=0                 #순위 표시
for tr in tr_soup:
    cnt+=1
    rank=cnt
    title=tr.find('div', class_="ellipsis rank01").get_text().replace('\n',"")
    artist=tr.find('span', class_="checkEllipsis").get_text().replace('\n',"")
    album=tr.find('div', class_="ellipsis rank03").get_text().replace('\n',"")
    starcnt=tr.find('span', class_="cnt").get_text().replace('\n총건수\n',"")
    
    melon_list.append({"순위":rank, "곡명":title, "가수":artist, '앨범':album, "좋아요":starcnt})


df2=pd.DataFrame(melon_list)
df2
    

,순위,곡명,가수,앨범,좋아요
0,1,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,"107,683"
1,2,Next Level,aespa,Next Level,"164,549"
2,3,Butter,방탄소년단,Butter,"223,985"
3,4,신호등,이무진,신호등,"110,609"
4,5,Permission to Dance,방탄소년단,Butter / Permission to Dance,"116,653"
...,...,...,...,...,...
95,96,Don't Start Now,Dua Lipa,Future Nostalgia,"142,129"
96,97,Love poem,아이유,Love poem,"306,293"
97,98,내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),규현 (KYUHYUN),내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),"106,850"
98,99,Hello Future,NCT DREAM,Hello Future - The 1st Album Repackage,"67,876"


In [91]:
tr.select("div.ellipsis.rank01")[0].get_text().replace("\n", "")

'밤이 되니까'

In [96]:
aa=melon_soup.select('tbody>tr')    # 태그 전체를 찾아 리스트형으로 제공, find_all()
aa[0].find('div', class_="ellipsis rank01")

<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19030101',33625988);" title="바라만 본다 재생">바라만 본다</a>
</span></div>